In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"C:\Users\dipay\Downloads\final_food_delivery_dataset.csv")
df['total_amount'] = pd.to_numeric(df['total_amount'], errors='coerce')
df['order_date'] = pd.to_datetime(df['order_date'], format='%d-%m-%Y')

In [5]:
print("\n1. GOLD MEMBERS REVENUE BY CITY")
gold_revenue_by_city = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().round(0)
print(gold_revenue_by_city.sort_values(ascending=False).astype(int).to_string())

print("\n2. AVERAGE ORDER VALUE BY CUISINE")
cuisine_avg_order = df.groupby('cuisine')['total_amount'].mean().round(2)
print(cuisine_avg_order.sort_values(ascending=False).to_string())

print("\n3. USERS BY TOTAL SPENDING (>₹1000 highlighted)")
user_total = df.groupby('user_id')['total_amount'].sum().round(0)
high_value_count = (user_total > 1000).sum()
print(f"Total users with >₹1000: {high_value_count}")
print(user_total.sort_values(ascending=False).head(10).astype(int).to_string())

print("\n4. REVENUE BY RATING RANGE")
df['rating_range'] = pd.cut(df['rating'], bins=[2.9, 3.5, 4.0, 4.5, 5.1], labels=['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0'], right=False)
rating_revenue = df.groupby('rating_range', observed=True)['total_amount'].sum().round(0)
print(rating_revenue.astype(int).sort_values(ascending=False).to_string())

print("\n5. GOLD MEMBERS AVG ORDER VALUE BY CITY")
gold_city_avg = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().round(2)
print(gold_city_avg.sort_values(ascending=False).to_string())

print("\n6. CUISINE: RESTAURANTS vs REVENUE")
cuisine_stats = df.groupby('cuisine').agg({'restaurant_id': 'nunique', 'total_amount': 'sum'}).round(0)
cuisine_stats.columns = ['Restaurants', 'Revenue']
print(cuisine_stats.sort_values('Revenue', ascending=False).astype(int).to_string())


1. GOLD MEMBERS REVENUE BY CITY
city
Chennai      1080910
Pune         1003012
Bangalore     994703
Hyderabad     896740

2. AVERAGE ORDER VALUE BY CUISINE
cuisine
Mexican    808.02
Italian    799.45
Indian     798.47
Chinese    798.39

3. USERS BY TOTAL SPENDING (>₹1000 highlighted)
Total users with >₹1000: 2544
user_id
1515    11556
650     10747
496      9634
2586     9487
2615     9237
636      9087
2973     9063
874      8917
1337     8908
2694     8876

4. REVENUE BY RATING RANGE
rating_range
4.6–5.0    2676078
3.0–3.5    1817951
4.1–4.5    1799541
3.6–4.0    1718055

5. GOLD MEMBERS AVG ORDER VALUE BY CITY
city
Chennai      808.46
Hyderabad    806.42
Bangalore    793.22
Pune         781.16

6. CUISINE: RESTAURANTS vs REVENUE
         Restaurants  Revenue
cuisine                      
Mexican          128  2085503
Italian          126  2024204
Indian           126  1971413
Chinese          120  1930505


In [6]:
print("\n7. MEMBERSHIP ORDER BREAKDOWN")
membership_stats = df['membership'].value_counts()
total_orders = len(df)
gold_pct = round((membership_stats['Gold'] / total_orders) * 100)
print(membership_stats.to_string())
print(f"Gold percentage: {gold_pct}%")

print("\n8. AVG ORDER (<20 orders)")
rest_avg = df.groupby('restaurant_id')['total_amount'].mean()
rest_count = df.groupby('restaurant_id').size()
specific_rests = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']
print("\nSPECIFIC RESTAURANTS DETAILS (<20 orders check):")
specific_details = []
for rest_name in specific_rests:
    matching = df[df['restaurant_name'].str.contains(rest_name, case=False, na=False)]
    if not matching.empty:
        rest_id = matching['restaurant_id'].iloc[0]
        avg_order = rest_avg.get(rest_id, 0)
        order_count = rest_count.get(rest_id, 0)
        specific_details.append({
            'Restaurant': rest_name,
            'Avg_Order': f"₹{avg_order:.2f}",
            'Order_Count': order_count,
            'Low_Volume': 'YES' if order_count < 20 else 'NO'
        })
    else:
        specific_details.append({'Restaurant': rest_name, 'Avg_Order': '-', 'Order_Count': 0, 'Low_Volume': '-'})
print(pd.DataFrame(specific_details).to_string(index=False))



7. MEMBERSHIP ORDER BREAKDOWN
membership
Regular    5013
Gold       4987
Gold percentage: 50%

8. AVG ORDER (<20 orders)

SPECIFIC RESTAURANTS DETAILS (<20 orders check):
                   Restaurant Avg_Order  Order_Count Low_Volume
           Grand Cafe Punjabi   ₹759.14           20         NO
Grand Restaurant South Indian   ₹842.57           29         NO
      Ruchi Mess Multicuisine   ₹938.20           18        YES
          Ruchi Foods Chinese   ₹686.60           19        YES


In [9]:
print("\n9. SPECIFIC MEMBERSHIP + CUISINE REVENUE COMBINATIONS")
specific_combos = [('Gold', 'Indian'), ('Gold', 'Italian'), ('Regular', 'Indian'), ('Regular', 'Chinese')]
combo_results = []
for membership, cuisine in specific_combos:
    revenue = df[(df['membership'] == membership) & (df['cuisine'] == cuisine)]['total_amount'].sum().round(0)
    combo_results.append({'Membership': membership, 'Cuisine': cuisine, 'Revenue': int(revenue)})
combo_df = pd.DataFrame(combo_results).sort_values('Revenue', ascending=False)
print(combo_df.to_string(index=False))
winner_row = combo_df.iloc[0]
print(f"\n✓ HIGHEST REVENUE: {winner_row['Membership']} + {winner_row['Cuisine']} (₹{winner_row['Revenue']:,.0f})")

print("\n10. QUARTERLY REVENUE")
df['quarter'] = df['order_date'].dt.to_period('Q')
quarter_revenue = df.groupby('quarter', observed=True)['total_amount'].sum().round(0)
print(quarter_revenue.astype(int).sort_values(ascending=False).to_string())


9. SPECIFIC MEMBERSHIP + CUISINE REVENUE COMBINATIONS
Membership Cuisine  Revenue
      Gold Italian  1005779
   Regular  Indian   992100
      Gold  Indian   979312
   Regular Chinese   952791

✓ HIGHEST REVENUE: Gold + Italian (₹1,005,779)

10. QUARTERLY REVENUE
quarter
2023Q3    2037385
2023Q4    2018264
2023Q1    1993425
2023Q2    1945349
2024Q1      17202
Freq: Q-DEC


In [10]:
print("\n1. TOTAL GOLD MEMBERSHIP ORDERS")
gold_orders = len(df[df['membership'] == 'Gold'])
print(f"Total Gold orders: {gold_orders:,}")

print("\n2. HYDERABAD TOTAL REVENUE")
hyderabad_revenue = df[df['city'] == 'Hyderabad']['total_amount'].sum().round(0)
print(f"Hyderabad revenue: ₹{int(hyderabad_revenue):,}")

print("\n3. DISTINCT USERS")
distinct_users = df['user_id'].nunique()
print(f"Distinct users: {distinct_users:,}")

print("\n4. GOLD MEMBERS AVERAGE ORDER VALUE")
gold_avg = df[df['membership'] == 'Gold']['total_amount'].mean().round(2)
print(f"Gold avg order value: ₹{gold_avg}")

print("\n5. ORDERS WITH RESTAURANT RATING ≥ 4.5")
high_rating_orders = len(df[df['rating'] >= 4.5])
print(f"High rating orders (≥4.5): {high_rating_orders:,}")

print("\n6. TOP GOLD REVENUE CITY & ORDERS")
gold_by_city = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum()
top_gold_city = gold_by_city.idxmax()
top_city_orders = len(df[(df['membership'] == 'Gold') & (df['city'] == top_gold_city)])
print(f"Top Gold revenue city: {top_gold_city}")
print(f"Revenue: ₹{int(gold_by_city.max()):,}")
print(f"Gold orders in top city: {top_city_orders:,}")


1. TOTAL GOLD MEMBERSHIP ORDERS
Total Gold orders: 4,987

2. HYDERABAD TOTAL REVENUE
Hyderabad revenue: ₹1,889,367

3. DISTINCT USERS
Distinct users: 2,883

4. GOLD MEMBERS AVERAGE ORDER VALUE
Gold avg order value: ₹797.15

5. ORDERS WITH RESTAURANT RATING ≥ 4.5
High rating orders (≥4.5): 3,374

6. TOP GOLD REVENUE CITY & ORDERS
Top Gold revenue city: Chennai
Revenue: ₹1,080,909
Gold orders in top city: 1,337
